# 9.2 Implement our own system assembling

In this tutorial we

* write an integrators for $\int_T f v dx$ and $\int_T \nabla u \nabla v dx$: <br>
[myIntegrator.hpp](/edit/myIntegrator.hpp)
[myIntegrator.cpp](/edit/myIntegrator.cpp)


* put together element matrices to the global system matrix: <br>
[myAssembling.cpp](/edit/myAssembling.cpp)

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import unit_square

from ngsolve.fem import CompilePythonModule
from pathlib import Path

m = CompilePythonModule(Path('myassemblemodule.cpp'), init_function_name='mymodule')
dir (m)

In [ ]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))
fes = H1(mesh, order=3, dirichlet=".*")
u, v = fes.TnT()

### use our own integrators for element matrix calculation:

In [ ]:
a = BilinearForm(fes)
a += m.MyLaplace(CF(1))
a.Assemble()

f = LinearForm(fes)
f += m.MySource(x)
f.Assemble();

In [ ]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
Draw (gfu);

### use our own matrix assembling function:

In [ ]:
for l in range(4): mesh.Refine()
fes.Update()
gfu.Update()
f.Assemble();

In [ ]:
with TaskManager(pajetrace=10**8):
    # using our integrator
    mymatrix = m.MyAssembleMatrix(fes, m.MyLaplace(CF(1)), parallel=True)
    
    # using NGSolve built-in symbolic integrator
    # integrator = BilinearForm(grad(u)*grad(v)*dx).integrators[0]
    # mymatrix = m.MyAssembleMatrix(fes, integrator, parallel=True)

# print ("my matrix = ", mymat)

if fes.ndof < 100000:
    gfu.vec.data = mymatrix.Inverse(fes.FreeDofs()) * f.vec
    Draw (gfu);

In [ ]:
from ngsolve.ngstd import Timers
for t in Timers(): 
    if t["name"].startswith("MyAssemble"): 
        print (t["name"],"   ", t["time"])